In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import librosa
import matplotlib.pyplot as plt
#from scipy.stats import pearsonr as p
from pathlib import Path
import time
#from sklearn.preprocessing import OneHotEncoder
import random
from IPython.display import Audio

import torch
import torch.nn as nn
#import torch.optim as optim
#import torch.nn.functional as F
#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
#from torch.cuda import amp
#from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau, OneCycleLR
#import pytorch_lightning as pl
#from pytorch_lightning.callbacks import ModelCheckpoint, BackboneFinetuning, EarlyStopping
import torchvision.transforms.functional as fn
#from sklearn.preprocessing import LabelEncoder

# Audio 
import torchaudio
from torchaudio.transforms import MelSpectrogram, Resample
import gc
from tqdm import tqdm

In [7]:
test_dir = Path('/kaggle/input/birdclef-2023/test_soundscapes')
sub_df = pd.read_csv('/kaggle/input/birdclef-2023/sample_submission.csv')
model_dir = "/kaggle/input/very-last/model_vit_low9_.bin"
train_metadata = pd.read_csv('/kaggle/input/birdclef-2023/train_metadata.csv')

In [8]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [9]:
sub_df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,soundscape_29201_10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,soundscape_29201_15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
class CFG:
    seed = 42
    DEBUG = True
    train_root = '/kaggle/input/birdclef-2023/train_audio/'
    Trans = False
    file_path = 'filepath'
    label = 'primary_label'
    sample_rate  =  32000 # torch audio default
    max_time = 5# must be set to 5 for Inference ? 
    window_size_frames = 5.0
    hop_size_frames = 5.0
    batch_size = 32
    normalize = True
    #Visualize = True
    num_classes = 264
    pretrained = False
    use_mixup = False

    Visualize = False
    weight_decay = 1e-3
    device = 'GPU'
    


In [11]:
class Create_Dataset(Dataset):
    '''
    df : dataframe
    source_column : the column with the text
    target_column : Column with the target
    transform : for possible augmentations
    freq_threshold: minimum amount of times a word needs to be appear in the corpus to be considered for vocab
    '''
    
    def __init__(self, df,arr, source_column = CFG.file_path, target_column = CFG.label,
                 transform = CFG.Trans, max_time = 5,#CFG.max_time,
                  sample_rate = CFG.sample_rate, step = None):
        
        self.df = df
        self.transform = transform
        print(source_column)
        self.source_audio = self.df[source_column]

        self.classes = CFG.num_classes
        self.sample_rate = sample_rate
        self.audio_length = int(max_time*sample_rate) 
        self.array = arr
        self.step =  self.audio_length
        
    def __len__(self):
        return len(self.df)
    
    #@staticmethod
    def normalize(self,image):
        image = image / 255.0
        image = torch.stack([image, image, image])
        return image
    
    # Isto vai-te buscar automaticamente uma linha do df e transforma-a em tokens + target
    # não precisas de inicializar, podes apenas chamar o objecto e ver o seu índice
    
    def __getitem__(self, index):
        #print(index)
        source_audio = self.source_audio[index]
        
        audio, sample_rate = torchaudio.load(source_audio)
        
        if self.array == 1:
            arr = [0,int((int(audio.shape[1]/(CFG.sample_rate*5))/4)*CFG.sample_rate*5)]
        elif self.array == 2:
            arr = [int((int(audio.shape[1]/(CFG.sample_rate*5))/4)*CFG.sample_rate*5),int((int(audio.shape[1]/(CFG.sample_rate*5))/4)*CFG.sample_rate*5)*2]
        elif self.array == 3:
            arr = [int((int(audio.shape[1]/(CFG.sample_rate*5))/4)*CFG.sample_rate*5)*2,int((int(audio.shape[1]/(CFG.sample_rate*5))/4)*CFG.sample_rate*5)*3]
        elif self.array == 4:
            arr = [int((int(audio.shape[1]/(CFG.sample_rate*5))/4)*CFG.sample_rate*5)*3,audio.shape[1]]
        
        audio = self.to_mono(audio)
        #print('antes: ', audio.shape)
        if CFG.par == True:
           
            audio = audio[arr[0]:arr[1]]
        
        #print('depois: ', audio.shape)
        if sample_rate != self.sample_rate:
            resample = Resample(sample_rate, self.sample_rate)
            audio = resample(audio)
          
        
        # Vais tirar bocados de áudio do áudio inicial com tamanhos = a step 
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
        
        
        images = [self.spectro_gram(sr = self.sample_rate,aud = audio) for audio in audios]
        
        images = np.stack(images)
        del audios, audio
        return images
   

    
    
    
    #@staticmethod
    def spectro_gram(self,sr ,aud, n_mels = 64, n_fft = 1024, hop_len=None, trans = False):
        sig = aud
        
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = torchaudio.transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        image = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(spec)
        #print(image.size())
        if trans == False:
            #print(image.shape)
            image = self.normalize(image)
            #print(image.shape)
            image = fn.resize(image, size=[224,224])
        return (image)
    
    def to_mono(self, audio):
        return torch.mean(audio, axis=0)

In [12]:
class Create_Dataset_2_cores(Dataset):
    '''
    df : dataframe
    source_column : the column with the text
    target_column : Column with the target
    transform : for possible augmentations
    freq_threshold: minimum amount of times a word needs to be appear in the corpus to be considered for vocab
    '''
    
    def __init__(self, df,arr, source_column = CFG.file_path, target_column = CFG.label,
                 transform = CFG.Trans, max_time = 5,#CFG.max_time,
                  sample_rate = CFG.sample_rate, step = None):
        
        self.df = df
        self.transform = transform
        print(source_column)
        self.source_audio = self.df[source_column]

        self.classes = CFG.num_classes
        self.sample_rate = sample_rate
        self.audio_length = int(max_time*sample_rate) 
        self.array = arr
        self.step =  self.audio_length
        
    def __len__(self):
        return len(self.df)
    
    #@staticmethod
    def normalize(self,image):
        image = image / 255.0
        image = torch.stack([image, image, image])
        return image
    
    # Isto vai-te buscar automaticamente uma linha do df e transforma-a em tokens + target
    # não precisas de inicializar, podes apenas chamar o objecto e ver o seu índice
    
    def __getitem__(self, index):
        #print(index)
        source_audio = self.source_audio[index]
        
        audio, sample_rate = torchaudio.load(source_audio)
        
        if self.array == 1:
            arr = [0,int((int(audio.shape[1]/(CFG.sample_rate*5))/2)*CFG.sample_rate*5)]
    
        elif self.array == 4:
            arr = [int((int(audio.shape[1]/(CFG.sample_rate*5))/2)*CFG.sample_rate*5),audio.shape[1]]
        
        audio = self.to_mono(audio)
        #print('antes: ', audio.shape)
        if CFG.par == True:
           
            audio = audio[arr[0]:arr[1]]
        
        #print('depois: ', audio.shape)
        if sample_rate != self.sample_rate:
            resample = Resample(sample_rate, self.sample_rate)
            audio = resample(audio)
          
       
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
        
        images = [self.spectro_gram(sr = self.sample_rate,aud = audio) for audio in audios]
        
        images = np.stack(images)
        del audios, audio
        return images
   

    
    
    
    #@staticmethod
    def spectro_gram(self,sr ,aud, n_mels = 64, n_fft = 1024, hop_len=None, trans = False):
        sig = aud
        
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = torchaudio.transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        image = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(spec)
        #print(image.size())
        if trans == False:
            #print(image.shape)
            image = self.normalize(image)
            #print(image.shape)
            image = fn.resize(image, size=[224,224])
        return (image)
    
    def to_mono(self, audio):
        return torch.mean(audio, axis=0)

In [13]:
def plot_random_spec_spectogram(dataGen, rand_number = 10):
    rand_numba = random.randint(0,len(dataGen))
    
    if CFG.Visualize:
        #rand_number = random.randint(0,119)
        print(rand_number,len(dataGen[0][0]))
        k = dataGen[0][rand_number][0]
        plt.figure(figsize=(14, 5))
        librosa.display.specshow(np.array(k,dtype = float), sr=CFG.sample_rate, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram Trans(dB)')
        
        g_audio = dataGen[0][rand_number][0]
        plt.figure(figsize=(14, 5))
        librosa.display.specshow(np.array(g_audio,dtype = float), sr=CFG.sample_rate, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Original Spectrogram (dB)')
        #print(dataGen[rand_numba][2])
        

    else:
        audio = dataGen[rand_numba][0][0]
        plt.figure(figsize=(14, 5))
        librosa.display.specshow(np.array(audio), sr=CFG.sample_rate, x_axis='time', y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram (dB)')

In [14]:
#CFG.Visualize = True
#CFG.par = False
#test = Create_Dataset(df_test,"")

In [15]:
#a = test[0]

In [16]:
#a.shape

In [17]:
#len(test)

In [18]:
#CFG.Trans = 'Nuone'
#CFG.Trans = None

In [19]:
#plot_random_spec_spectogram(test,1)

In [20]:
CFG.Visualize = False

In [21]:
class ViTBase16(nn.Module):
    def __init__(self,  num_classes = CFG.num_classes,
                 pretrained = CFG.pretrained):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224_in21k", pretrained=pretrained)
     
        
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [22]:
print("Model Creation")
model = ViTBase16()
checkpoint = torch.load(model_dir, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)

Model Creation


<All keys matched successfully>

In [23]:
df_test = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in test_dir.glob("*.ogg")],
    columns = ["filename", "name" ,"id", "filepath"]
)
print(df_test.shape)
df_test.head()

(1, 4)


,filename,name,id,filepath
0,soundscape_29201,soundscape,29201,/kaggle/input/birdclef-2023/test_soundscapes/s...


In [24]:
import concurrent.futures

In [25]:
def predict(arr):
    dataset_test = Create_Dataset(df_test,arr)
    #model.to('cpu')
    model.eval()    
    predictions = []
    for en in range(len(dataset_test)):
        
        images = torch.from_numpy(dataset_test[en])
        print(images.shape)
        with torch.no_grad():
            outputs = model(images).sigmoid().detach().cpu().numpy()
            #outputs = model(images).detach().cpu().numpy()
            print('Output shape : ',outputs.shape)
            #print('Model image : ',model(images).shape )
#             pred_batch.extend(outputs.detach().cpu().numpy())
#         pred_batch = np.vstack(pred_batch)
        predictions.append(outputs)
            
    
    return predictions

In [26]:
def predict_2_cores(arr):
    dataset_test = Create_Dataset_2_cores(df_test,arr)
    #model.to('cpu')
    model.eval()    
    predictions = []
    for en in range(len(dataset_test)):
        
        images = torch.from_numpy(dataset_test[en])
        print(images.shape)
        with torch.no_grad():
            outputs = model(images).sigmoid().detach().cpu().numpy()
            #outputs = model(images).detach().cpu().numpy()
            print('Output shape : ',outputs.shape)
            #print('Model image : ',model(images).shape )
#             pred_batch.extend(outputs.detach().cpu().numpy())
#         pred_batch = np.vstack(pred_batch)
        predictions.append(outputs)
            
    
    return predictions

In [27]:
CFG.par = True
#CFG.Trans = None

In [28]:
%%time


print("Running Inference..")

if CFG.par == True:
    n_cores = os.cpu_count()
    n_cores
    if n_cores == 4:
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            #dicts = list(executor.map(prediction_for_clip, all_audios))
            preds = np.array(list(executor.map(predict,[1,2,3,4])))
        preds = np.concatenate((preds[0],preds[1],preds[2],preds[3]),axis = 1)
    elif n_cores == 2:
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            #dicts = list(executor.map(prediction_for_clip, all_audios))
            preds = np.array(list(executor.map(predict_2_cores,[1,2])))
        preds = np.concatenate((preds[0],preds[1]),axis = 1)
    else: 
        CFG.par = False
        preds = predict("")
else:
    preds = predict("")   

gc.collect()
torch.cuda.empty_cache()

Running Inference..
filepath
filepath
filepath
filepath


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([30, 3, 224, 224])
torch.Size([30, 3, 224, 224])
torch.Size([30, 3, 224, 224])
torch.Size([30, 3, 224, 224])
Output shape :  (30, 264)
Output shape :  (30, 264)
Output shape :  (30, 264)
Output shape :  (30, 264)
CPU times: user 2min 47s, sys: 22 s, total: 3min 9s
Wall time: 59.6 s


In [29]:
#np.concatenate((preds[0],preds[1],preds[2],preds[3]),axis = 1).shape

In [30]:
filenames = df_test.filename.values.tolist()

bird_cols = list(pd.get_dummies(train_metadata['primary_label']).columns)
sub_df = pd.DataFrame(columns=['row_id']+bird_cols)

In [31]:
for i, file in enumerate(filenames):
    pred = preds[i]
    num_rows = len(pred)
    row_ids = [f'{file}_{(i+1)*5}' for i in range(num_rows)]
    df = pd.DataFrame(columns=['row_id']+bird_cols)
    
    df['row_id'] = row_ids
    df[bird_cols] = pred
    
    sub_df = pd.concat([sub_df,df]).reset_index(drop=True)
    

In [32]:
sub_df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.672704,0.674780,0.710523,0.666238,0.132109,0.868356,0.711755,0.153575,0.595431,...,0.085558,0.194814,0.648310,0.160931,0.081219,0.405500,0.555928,0.108136,0.465635,0.643953
1,soundscape_29201_10,0.918386,0.663636,0.364458,0.827652,0.025846,0.190199,0.283346,0.600401,0.772332,...,0.303001,0.451711,0.902978,0.836101,0.105958,0.131861,0.111469,0.473670,0.353403,0.264374
2,soundscape_29201_15,0.753482,0.741929,0.600111,0.753363,0.034621,0.403684,0.184427,0.611486,0.343918,...,0.024531,0.844772,0.877839,0.369374,0.025576,0.083334,0.065432,0.623052,0.774909,0.505265
3,soundscape_29201_20,0.215242,0.583343,0.572503,0.942814,0.136594,0.712793,0.639659,0.300803,0.854941,...,0.033745,0.706491,0.178416,0.195271,0.034514,0.148922,0.367139,0.062103,0.807713,0.743812
4,soundscape_29201_25,0.801042,0.446160,0.468106,0.990659,0.191527,0.642555,0.269996,0.918668,0.948059,...,0.044748,0.844415,0.287007,0.296736,0.041793,0.152515,0.214128,0.130934,0.817004,0.437339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.919840,0.615138,0.178952,0.870141,0.061838,0.311638,0.363901,0.183645,0.831690,...,0.065708,0.818480,0.416169,0.488839,0.293322,0.173757,0.215011,0.177407,0.502654,0.200081
116,soundscape_29201_585,0.125440,0.410568,0.066915,0.022578,0.004862,0.382540,0.959527,0.267655,0.943893,...,0.271971,0.534871,0.268316,0.062123,0.956299,0.353353,0.302196,0.678091,0.415690,0.214422
117,soundscape_29201_590,0.789834,0.580795,0.572230,0.772386,0.093577,0.412385,0.534022,0.211962,0.983624,...,0.041594,0.539700,0.091197,0.355077,0.541654,0.037960,0.220519,0.208035,0.786210,0.506397
118,soundscape_29201_595,0.787753,0.531526,0.451534,0.886570,0.063801,0.500575,0.181038,0.921998,0.959904,...,0.017096,0.865478,0.206361,0.792045,0.442518,0.117631,0.185783,0.175223,0.894282,0.610030


In [33]:
"""for indx, row in sub_df.iterrows():
    #print(indx)
    #print(max(row[1:]))
    #print(min(row[1:]))
    for ind, a in enumerate(sub_df.iloc[indx][1:].to_list()):
        b = (a- min(row[1:]))/(max(row[1:]) - min(row[1:]))
        #print(b)
        sub_df.loc[indx][1:][ind] = b"""

'for indx, row in sub_df.iterrows():\n    #print(indx)\n    #print(max(row[1:]))\n    #print(min(row[1:]))\n    for ind, a in enumerate(sub_df.iloc[indx][1:].to_list()):\n        b = (a- min(row[1:]))/(max(row[1:]) - min(row[1:]))\n        #print(b)\n        sub_df.loc[indx][1:][ind] = b'

In [34]:
sub_df.to_csv('submission.csv',index=False)

In [35]:
import multiprocessing

In [36]:
n_cores = multiprocessing.cpu_count()
n_cores

4

In [37]:
n_cores = os.cpu_count()
n_cores

4